In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../projects/skimpy_bench/src/")
sys.path.insert(0, "../projects/skimpy_blox/")

In [2]:
import functools
import time
import traceback
from pathlib import Path
from skimpy_blox.minecraft import SkimpyMinecraftLevel
import pandas as pd

In [3]:
SkimpyMinecraftLevel.from_world_infer_order("/home/tdimson/Downloads/Vanillaburp's Dinner Party", num_chunks=1000)


{(0, 1, 2): 32.0, (0, 2, 1): 512.0, (1, 0, 2): 2.0, (1, 2, 0): 512.0, (2, 0, 1): 2.0, (2, 1, 0): 32.0}


In [4]:
def process_folder(input_folder, pickle_folder):
    input_folder = Path(input_folder)
    pickle_folder = Path(pickle_folder)
    for d in (x for x in input_folder.iterdir() if input_folder.is_dir()):
        output_name = f"{d.name}.pickle.gz"
        output_path = pickle_folder / output_name
        
        if output_path.exists():
            print(f"Skipping {output_path} since it already exists")
            continue
        
        print(f"Processing {d.name} into {output_path}")
        start = time.time()
        try:
            level = SkimpyMinecraftLevel.from_world_infer_order(d, num_chunks=1000)
        except Exception:
            traceback.print_exc()
            continue
        level.dump(output_path)
        print(f"Finished {d.name} in {time.time() - start}s - compression ratio {level.rle_compression_ratio()}")

In [3]:
def stats_dataframe(pickle_folder):
    pickle_folder = Path(pickle_folder)
    rows = []
    for file in pickle_folder.glob("*.gz"):
        try:
            level = SkimpyMinecraftLevel.load(file)
        except EOFError:
            print(f"Error loading {file}")
            continue
        num_voxels = functools.reduce(lambda x, y: x * y, level.dense_dimensions())
        num_nonzero_voxels = level.num_nonzero_voxels()
        num_runs = level.num_runs()
        minecraft_representation_size = level.minecraft_representation_size()
        
        
        rows.append({
            "name": str(file.name).replace(".pickle.gz", ""),
            "width": level.dense_dimensions()[0],
            "height": level.dense_dimensions()[1],
            "depth": level.dense_dimensions()[2],
            "column_order": level.column_order,
            "num_voxels": num_voxels,
            "num_nonzero_voxels": num_nonzero_voxels,
            "num_runs": num_runs,
            "num_minecraft_representation": minecraft_representation_size,
            "rle_compression_full_dense": num_voxels / (2 *num_runs),
            "rle_compression_partial_dense": minecraft_representation_size / (2 * num_runs),
            "sparse_compression_full_dense": num_voxels / (2 * num_nonzero_voxels),
            "sparse_compression_partial_dense": minecraft_representation_size / (2 * num_nonzero_voxels), 
        })
        
    
    return pd.DataFrame(rows, columns=rows[0].keys())

stats = stats_dataframe("/mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled")
    

In [4]:
stats

,name,width,height,depth,column_order,num_voxels,num_nonzero_voxels,num_runs,num_minecraft_representation,rle_compression_full_dense,rle_compression_partial_dense,sparse_compression_full_dense,sparse_compression_partial_dense
0,SGC 3.0 BY West build Team,3002960,256,3002464,"(0, 2, 1)",2308167499120640,539009124,9444158,822501376,1.222008e+08,43.545511,2.141121e+06,0.762975
1,PvP Arena,2560,80,2160,"(0, 2, 1)",442368000,2301004,169723,9043968,1.303206e+03,26.643319,9.612500e+01,1.965222
2,Los Angelcraft by Fujiwara1990,1520,256,1696,"(1, 2, 0)",659947520,165056808,8953896,204070912,3.685253e+01,11.395649,1.999153e+00,0.618184
3,MinePark1.0,1648,256,1696,"(1, 2, 0)",715522048,106499949,5562755,136843264,6.431364e+01,12.299954,3.359260e+00,0.642457
4,Union Islands,8944,256,6112,"(0, 2, 1)",13994426368,3062810912,175499995,3793580032,3.987016e+01,10.807921,2.284572e+00,0.619297
5,SmithyMC Jurassic World 24-10-15 Minecraft 1.8...,7936,256,12688,"(1, 2, 0)",25777143808,1421177596,153375525,1695944704,8.403278e+01,5.528733,9.068938e+00,0.596669
6,worldX,8352,240,8432,"(0, 2, 1)",16901775360,4373518629,169955013,4778450944,4.972426e+01,14.057988,1.932286e+00,0.546294
7,Piston House III(ByEuclides),592,112,640,"(1, 2, 0)",42434560,21115197,2636057,24985600,8.048870e+00,4.739199,1.004835e+00,0.591650
8,Ironman by McMakistein,7328,256,10816,"(1, 2, 0)",20290469888,23546264,1512153,596058112,6.709133e+03,197.089220,4.308639e+02,12.657170
9,YandereSim(Better),1040,48,1120,"(1, 2, 0)",55910400,1561360,32605,6504448,8.573900e+02,99.746174,1.790439e+01,2.082943


In [14]:
set(stats[stats["num_voxels"] < 2 ** 31 - 1]["name"])

{'Heist v1.4b',
 'Hogwarts1.0',
 'Hydra Vanilla Mod Pack',
 'Isles Of Arqualis',
 'Los Angelcraft by Fujiwara1990',
 'MinePark1.0',
 'Modern House',
 'Piston House II(ByEuclides)',
 'Piston House III(ByEuclides)',
 'PvP Arena',
 'Steven Universe',
 'StrandedRaft',
 'YandereSim(Better)',
 'ZeSpawn'}

In [13]:
stats["rle_compression_vs_sparse"] =stats["num_nonzero_voxels"] /  stats["num_runs"] 


In [10]:
rle_size = 2 * stats["num_runs"]


In [14]:
def print_stats(stats):
    print(f"Num items: {len(stats)}")
    print(f"Median num voxels: {stats['num_voxels'].median()}")
    print(f"Median non-zero voxels: {stats['num_nonzero_voxels'].median()}")
    typical_voxel = stats["num_voxels"].sort_values().iloc[len(stats) // 2 ]
    example_for_dim = stats[stats["num_voxels"] == typical_voxel].iloc[0]
    print(f"Dimension for Median Num Voxels: {example_for_dim.width}x{example_for_dim.height}x{example_for_dim.depth}")
    print(f"Median RLE Compression Ratio Vs. Dense: {stats['rle_compression_full_dense'].median()}x")
    print(f"Median RLE Compression Ratio Vs. Sparse: {stats['rle_compression_vs_sparse'].median()}x")
    print(f"Min RLE Compression Ratio Vs. Dense: {stats['rle_compression_full_dense'].min()}x")
    print(f"Max RLE Compression Ratio Vs. Dense: {stats['rle_compression_full_dense'].max()}x")
    print(f"Median Sparse Compression Ratio Vs. Dense: {stats['sparse_compression_full_dense'].median()}x")
    print(f"Median RLE Compression Ratio Vs. Mixed: {stats['rle_compression_partial_dense'].median()}x")
    print(f"Median Sparse Compression Ratio Vs. Mixed: {stats['sparse_compression_partial_dense'].median()}x")
    
print_stats(stats)

Num items: 25
Median num voxels: 1617100800.0
Median non-zero voxels: 106499949.0
Dimension for Median Num Voxels: 1600x224x4512
Median RLE Compression Ratio Vs. Dense: 58.68888384266351x
Median RLE Compression Ratio Vs. Sparse: 13.678428454182818x
Min RLE Compression Ratio Vs. Dense: 8.048869959943962x
Max RLE Compression Ratio Vs. Dense: 122200809.17328152x
Median Sparse Compression Ratio Vs. Dense: 3.359260049974296x
Median RLE Compression Ratio Vs. Mixed: 12.299954249288348x
Median Sparse Compression Ratio Vs. Mixed: 0.6265783522657582x


In [16]:
stats[stats["name"] == "Hogwarts1.0"]

,name,width,height,depth,column_order,num_voxels,num_nonzero_voxels,num_runs,num_minecraft_representation,rle_compression_full_dense,rle_compression_partial_dense,sparse_compression_full_dense,sparse_compression_partial_dense,rle_compression_vs_sparse
17,Hogwarts1.0,2448,256,1088,"(0, 2, 1)",681836544,12753121,1433678,56594432,237.792776,19.737498,26.732144,2.218846,8.895387


In [ ]:
stats.sort_values("rle_compression_partial_dense")

In [ ]:
level = SkimpyMinecraftLevel.load("/mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/YandereSim(Better).pickle.gz")

In [ ]:
level.megatensor()

In [5]:
process_folder(
    "/mnt/evo/projects/cs348k/Minecraft Levels - 2015/Uncompressed", 
    "/mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled"
)

Skipping /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/MinePark1.0.pickle.gz since it already exists
Skipping /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/Los Angelcraft by Fujiwara1990.pickle.gz since it already exists
Skipping /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/Ironman by McMakistein.pickle.gz since it already exists
Skipping /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/SmithyMC Jurassic World 24-10-15 Minecraft 1.8 Video 18.04.17.pickle.gz since it already exists
Skipping /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/Hogwarts1.0.pickle.gz since it already exists
Processing Yandere Sim Map into /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/Yandere Sim Map.pickle.gz



Skipping /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/Chume Labs 2 - Mapa.v1.pickle.gz since it already exists
Processing Newisle v1.2 into /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/Newisle v1.2.pickle.gz


Traceback (most recent call last):
  File "<ipython-input-4-268775f9415a>", line 15, in process_folder
    level = SkimpyMinecraftLevel.from_world_infer_order(d, num_chunks=1000)
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 146, in from_world_infer_order
    best_order, stats = cls.approx_best_column_order(world_folder, num_chunks)
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 139, in approx_best_column_order
    num_chunks=num_chunks).rle_compression_ratio()
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 169, in from_world
    for i, chunk in enumerate(tqdm(world.iter_chunks(), total=world.chunk_count())):
  File "/home/tdimson/anaconda3/envs/skimpy/lib/python3.7/site-packages/tqdm/notebook.py", line 218, in __iter__
    for obj in super(tqdm_notebook, self).__iter__(*args, **kwargs):
  File "/home/tdimson/anaconda3/envs/skimpy/lib/python3.7/site-packages/tqdm/std.py", line 1129, in __iter__
    for obj in iterable:
  File "../proj


{(0, 1, 2): 5.8386509765861945, (0, 2, 1): 5.938930840440737, (1, 0, 2): 4.061105198955637, (1, 2, 0): 5.9375686097897376, (2, 0, 1): 4.061105198955637, (2, 1, 0): 5.842583747565435}



Processing Jurassic Park into /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/Jurassic Park.pickle.gz


...

Traceback (most recent call last):
  File "<ipython-input-4-268775f9415a>", line 15, in process_folder
    level = SkimpyMinecraftLevel.from_world_infer_order(d, num_chunks=1000)
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 148, in from_world_infer_order
    return cls.from_world(world_folder, column_order=best_order)
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 185, in from_world
    chunk_list.append(cls.chunk_to_skimpy((x, y, z), chunk, column_order=column_order))
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 259, in chunk_to_skimpy
    numpy_chunk = cls.chunk_to_numpy(coord, chunk, column_order)
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 253, in chunk_to_numpy
    arr[cls._xyz_to_numpy_col((x, y, z), column_order)] = block_int_id
ValueError: invalid literal for int() with base 10: 'air'



Processing worldX into /mnt/evo/projects/cs348k/Minecraft Levels - 2015/Pickled/worldX.pickle.gz


Traceback (most recent call last):
  File "<ipython-input-4-268775f9415a>", line 15, in process_folder
    level = SkimpyMinecraftLevel.from_world_infer_order(d, num_chunks=1000)
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 146, in from_world_infer_order
    best_order, stats = cls.approx_best_column_order(world_folder, num_chunks)
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 139, in approx_best_column_order
    num_chunks=num_chunks).rle_compression_ratio()
  File "../projects/skimpy_blox/skimpy_blox/minecraft.py", line 169, in from_world
    for i, chunk in enumerate(tqdm(world.iter_chunks(), total=world.chunk_count())):
  File "/home/tdimson/anaconda3/envs/skimpy/lib/python3.7/site-packages/tqdm/notebook.py", line 218, in __iter__
    for obj in super(tqdm_notebook, self).__iter__(*args, **kwargs):
  File "/home/tdimson/anaconda3/envs/skimpy/lib/python3.7/site-packages/tqdm/std.py", line 1129, in __iter__
    for obj in iterable:
  File "../proj


{(0, 1, 2): 15.04228807303736, (0, 2, 1): 16.477559583314058, (1, 0, 2): 5.799017876293754, (1, 2, 0): 15.580246256138336, (2, 0, 1): 5.799017876293754, (2, 1, 0): 14.275358278034908}



Finished worldX in 14418.652968645096s - compression ratio 14.057987639352538


In [ ]:
level = SkimpyMinecraftLevel.from_world("/home/tdimson/Downloads/Vanillaburp's Dinner Party")

In [ ]:
level.dump("/home/tdimson/Downloads/vanillaburb.pickle.gz")